In [1]:

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Load your text data and preprocess it
# Example: text = open('your_text_file.txt', 'r').read().lower()
filepath = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text = open(filepath, 'r').read().lower()

# Define parameters and preprocess text
SEQ_LENGTH = 40
STEP_SIZE = 3

# Prepare sequences and next characters
sentences = []
next_characters = []

for i in range(0, len(text) - SEQ_LENGTH, STEP_SIZE):
    sentences.append(text[i: i + SEQ_LENGTH])
    next_characters.append(text[i + SEQ_LENGTH]) # Fixed indentation

characters = sorted(set(text))
char_to_index = {char: i for i, char in enumerate(characters)}
index_to_char = {i: char for i, char in enumerate(characters)}

# Prepare X and y datasets
X = np.zeros((len(sentences), SEQ_LENGTH, len(characters)), dtype=np.bool_)
y = np.zeros((len(sentences), len(characters)), dtype=np.bool_)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_to_index[char]] = 1
    y[i, char_to_index[next_characters[i]]] = 1 # Fixed indentation

# Define the model architecture (example: LSTM-based)
model = Sequential([
    LSTM(128, input_shape=(SEQ_LENGTH, len(characters))),
    Dense(len(characters), activation='softmax')
])

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

# Train the model (example: with 10 epochs)
model.fit(X, y, batch_size=128, epochs=10)

# Save the trained model
model.save('textgenerator.model')

# Now you can use this model for text generation

1115394/1115394 [==============================] - 0s 0us/step
Epoch 1/10
2905/2905 [==============================] - 332s 114ms/step - loss: 2.3623
Epoch 2/10
2905/2905 [==============================] - 323s 111ms/step - loss: 2.0140
Epoch 3/10
2905/2905 [==============================] - 319s 110ms/step - loss: 1.8794
Epoch 4/10
2905/2905 [==============================] - 319s 110ms/step - loss: 1.7878
Epoch 5/10
2905/2905 [==============================] - 321s 111ms/step - loss: 1.7192
Epoch 6/10
2905/2905 [==============================] - 320s 110ms/step - loss: 1.6661
Epoch 7/10
2905/2905 [==============================] - 322s 111ms/step - loss: 1.6243
Epoch 8/10
2905/2905 [==============================] - 319s 110ms/step - loss: 1.5905
Epoch 9/10
2905/2905 [==============================] - 319s 110ms/step - loss: 1.5617
Epoch 10/10
2905/2905 [==============================] - 322s 111ms/step - loss: 1.5368


In [2]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import Sequential

# Load your trained model
model = tf.keras.models.load_model('textgenerator.model')

# Text preprocessing - Use the same text file as you used for training
filepath = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text = open(filepath, 'r').read().lower()  # Use the same preprocessing as training

SEQ_LENGTH = 40
STEP_SIZE = 3

# Instead of subsetting, use the entire text for character set
characters = sorted(set(text))  # Use the full text to get all characters
char_to_index = {c: i for i, c in enumerate(characters)}
index_to_char = {i: c for i, c in enumerate(characters)}

# Now you can subset for generating sentences if you want
text = text[30000:80000]
sentences = []
next_characters = []
for i in range(0, len(text) - SEQ_LENGTH, STEP_SIZE):
    sentences.append(text[i: i + SEQ_LENGTH])
    next_characters.append(text[i + SEQ_LENGTH])

X = np.zeros((len(sentences), SEQ_LENGTH, len(characters)), dtype=bool)
y = np.zeros((len(sentences), len(characters)), dtype=bool)

for i, sentence in enumerate(sentences):
    for t, character in enumerate(sentence):
        X[i, t, char_to_index[character]] = 1
        y[i, char_to_index[next_characters[i]]] = 1

# Function to sample the next character
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# Function to generate text
def generate_text(length, temperature):
    start_index = random.randint(0, len(text) - SEQ_LENGTH - 1)
    generated = ''
    sentence = text[start_index: start_index + SEQ_LENGTH]
    generated += sentence
    for i in range(length):
        x_pred = np.zeros((1, SEQ_LENGTH, len(characters)))
        for t, character in enumerate(sentence):
            x_pred[0, t, char_to_index[character]] = 1

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, temperature)
        next_character = index_to_char[next_index]

        generated += next_character
        sentence = sentence[1:] + next_character
    return generated

# Generate text and display the output
print('-------0.2--------')
print(generate_text(300, 0.2))
print('-------0.4--------')
print(generate_text(300, 0.4))
print('-------0.6--------')
print(generate_text(300, 0.6))
print('-------0.8--------')
print(generate_text(300, 0.8))
print('-------1--------')
print(generate_text(300, 1.0))

-------0.2--------
 him not to be their bedfellow.
worthy come, my lord, and so man a good so longer.

second murderer:
sir, that i will be so more than you so son.

prince:
the come to the word the world be so more than he hath send
and some heart of the brother to the best.

cominius:
why, so do man of the great of his son.

cominius:
why, sir, if thou su
-------0.4--------
n when the navel of the state was touch'd the dest.

comentio:
stand the cares of his thousand his son.

second caurel:
o, that he that i am contring and to have
be more the breast be the behold to every
the now a part of my soul of hath such and mening
so meat be worth the consed of the bester
that brother her sweet and so a grave a prin
-------0.6--------
will owe another.

coriolanus:
on fair gors; and he have first, that hath death
the great and love to death, and thou sown
the body refort that hath the grace for yoursel,
and for their can very deach of the son
that the world and love against them be to'th
wil